In [1]:
pip install streamlit


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install jupyter-server-proxy


Note: you may need to restart the kernel to use updated packages.


In [3]:
%cd C:/Users/shady/llm_chatbot_project/backend

C:\Users\shady\llm_chatbot_project\backend


In [7]:
%run config.py


In [7]:
import os
import requests
import threading
from flask import Flask, request, jsonify
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
HF_API_KEY = os.getenv("HF_API_KEY")

# Prevent duplicate Flask app instances in Jupyter Notebook
if 'app' in globals():
    del app

# Initialize Flask app
app = Flask(__name__)

# Define the Hugging Face API model URL
MODEL_URL = "https://api-inference.huggingface.co/models/google/flan-t5-large"

@app.route('/chat', methods=['POST'])
def chat():
    try:
        # Get user input
        data = request.get_json()
        user_message = data.get('message', '')

        # Prepare request payload
        payload = {
            "inputs": f"User: {user_message}\nBot:",
            "parameters": {
                "max_length": 200,
                "temperature": 0.7,
                "top_p": 0.9,
                "repetition_penalty": 1.2
            }
        }

        # Send request to Hugging Face API
        headers = {"Authorization": f"Bearer {HF_API_KEY}"}
        response = requests.post(MODEL_URL, headers=headers, json=payload)

        # Process response
        if response.status_code == 200:
            try:
                response_json = response.json()
                if isinstance(response_json, list) and len(response_json) > 0:
                    bot_response = response_json[0].get("generated_text", "No response")
                else:
                    bot_response = "Unexpected response format"
            except Exception:
                bot_response = "Failed to parse response"
        else:
            bot_response = f"Error {response.status_code}: {response.text}"

        return jsonify({"response": bot_response}), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Run Flask in a background thread
def run_flask():
    app.run(host="0.0.0.0", port=8000, debug=False)

threading.Thread(target=run_flask).start()


 * Serving Flask app '__main__'
 * Debug mode: off


In [9]:
import requests

test_input = {"message": "where is france ?"}
response = requests.post("http://127.0.0.1:8000/chat", json=test_input)

print("Status Code:", response.status_code)
print("Response:", response.json())

127.0.0.1 - - [23/Feb/2025 04:21:49] "POST /chat HTTP/1.1" 200 -


Status Code: 200
Response: {'response': 'France is located in Europe'}


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501
